In [1]:
# check if tensorflow uses GPU
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 642844190171084316
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4163895296
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15476079312374875216
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
import pandas as pd

In [4]:
train_variants_df = pd.read_csv("../../data/training_variants/training_variants")
train_txt_df = pd.read_csv("../../data/training_text/training_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
train_data = pd.merge(train_variants_df, train_txt_df, on='ID')

# get rid of NaN entries in Text column
train_data = train_data[train_data['Text'].notnull()].copy()

In [5]:
train_data.head()

,ID,Gene,Variation,Class,Text
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...


In [6]:
# create an LSTM model to predict the class of a given text

# import libraries  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

print("Cleaning text...")

# create a function to clean the text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

# clean the text
train_data['Text'] = train_data['Text'].apply(lambda x: clean_text(x))

# create a function to remove stopwords
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return " ".join(filtered_text)

# remove stopwords
train_data['Text'] = train_data['Text'].apply(lambda x: remove_stopwords(x))

# create a function to lemmatize the text
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    word_tokens = word_tokenize(text)
    lemmatized_text = [lemmatizer.lemmatize(word) for word in word_tokens]
    return " ".join(lemmatized_text)

# lemmatize the text    
train_data['Text'] = train_data['Text'].apply(lambda x: lemmatize_text(x))

# create a function to stem the text
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def stem_text(text):
    word_tokens = word_tokenize(text)
    stemmed_text = [stemmer.stem(word) for word in word_tokens]
    return " ".join(stemmed_text)

# stem the text
train_data['Text'] = train_data['Text'].apply(lambda x: stem_text(x))

# create a function to remove punctuation
import string

def remove_punctuation(text):
    word_tokens = word_tokenize(text)
    no_punct_text = [word for word in word_tokens if word not in string.punctuation]
    return " ".join(no_punct_text)

# remove punctuation
train_data['Text'] = train_data['Text'].apply(lambda x: remove_punctuation(x))

# create a function to remove numbers
import re

def remove_numbers(text):
    text = re.sub(r'\d+', '', text)
    return text

# remove numbers
train_data['Text'] = train_data['Text'].apply(lambda x: remove_numbers(x))


print("Creating tokenizer...")


Cleaning text...


ModuleNotFoundError: No module named 'nltk'

In [8]:
# train the model
import os

os.environ["SM_FRAMEWORK"] = "tf.keras"

import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding


model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(keras.layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(keras.layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(keras.layers.Dense(10))

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          64000     
                                                                 
 lstm (LSTM)                 (None, 128)               98816     
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
Total params: 164,106
Trainable params: 164,106
Non-trainable params: 0
_________________________________________________________________


In [10]:
# train the model
import os

os.environ["SM_FRAMEWORK"] = "tf.keras"

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

UnimplementedError: Graph execution error:

Detected at node 'sequential_1/Cast' defined at (most recent call last):
    File "c:\Users\ibrah\anaconda3\envs\CSDS458_Bio\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\ibrah\anaconda3\envs\CSDS458_Bio\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\ibrah\anaconda3\envs\CSDS458_Bio\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\ibrah\anaconda3\envs\CSDS458_Bio\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\ibrah\anaconda3\envs\CSDS458_Bio\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\ibrah\anaconda3\envs\CSDS458_Bio\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\